In [72]:
# Célula 1: Importações e Configurações Iniciais
import time
import os
import requests # Ainda útil para baixar arquivos de links diretos se aparecerem
import xml.etree.ElementTree as ET # Para extrair o número da nota do XML
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import shutil # Adicione esta importação no topo da Célula 1 ou Célula 2

# --- Configurações para o Teste ---
MEUDANFE_WEB_URL = "https://www.meudanfe.com.br/"
SELENIUM_HEADLESS = False # Mude para False para VER o navegador abrindo e agindo.
                          # Mude para True quando o código estiver funcionando bem.
REQUEST_TIMEOUT_SECONDS = 30 # Tempo limite para carregamento de página/elementos

# Caminho para downloads temporários do Selenium
TEMP_DOWNLOAD_DIR = os.path.abspath("temp_downloads_selenium")
if not os.path.exists(TEMP_DOWNLOAD_DIR):
    os.makedirs(TEMP_DOWNLOAD_DIR)

# --- Função de Log Simplificada para o Notebook ---
def log_message(level, message):
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    log_output = f"[{timestamp}] [{level.upper()}] {message}"
    print(log_output)
    # Opcional: Salvar em um arquivo de log temporário no diretório do notebook
    with open(os.path.join("temp_logs_notebook.txt"), "a", encoding="utf-8") as f:
        f.write(log_output + "\n")

log_message("info", "Configurações e importações carregadas para o teste de scraping.")

# --- Função para extrair número da nota (reutilizada do transform.py) ---
def extract_note_number_from_xml(xml_content, logger_func):
    """
    Extrai o número da nota fiscal (nNF) do conteúdo XML.
    Assume que o XML é uma NF-e padrão.
    """
    try:
        root = ET.fromstring(xml_content)
        
        n_nf_element = None
        for elem in root.iter():
            if elem.tag.endswith('nNF'): # Procura por tags que terminam com 'nNF'
                n_nf_element = elem
                break 
        
        if n_nf_element is not None and n_nf_element.text:
            return n_nf_element.text.strip()

        nfe_namespace = {'nfe': 'http://www.portalfiscal.inf.br/nfe'} 
        
        nfe_node = root.find('.//nfe:NFe', nfe_namespace) 
        if nfe_node:
            inf_nfe_node = nfe_node.find('nfe:infNFe', nfe_namespace) 
            if inf_nfe_node:
                ide_node = inf_nfe_node.find('nfe:ide', nfe_namespace) 
                if ide_node:
                    n_nf_element = ide_node.find('nfe:nNF', nfe_namespace) 
                    if n_nf_element is not None and n_nf_element.text:
                        return n_nf_element.text.strip()

        logger_func("warning", "Não foi possível encontrar a tag 'nNF' no XML.")
        return None
    except ET.ParseError as e:
        logger_func("error", f"Erro ao parsear XML: {e}")
        return None
    except Exception as e:
        logger_func("error", f"Erro inesperado ao extrair número da nota do XML: {e}")
        return None

[2025-06-26 16:08:50] [INFO] Configurações e importações carregadas para o teste de scraping.


In [ ]:
# Célula 2: Inicializar o WebDriver (Chrome) - APRIMORADA PARA DOWNLOADS!

import os
import shutil # Para limpar a pasta de downloads entre os testes
import time # Para pausar e dar tempo para o arquivo ser baixado

# Certifique-se de que TEMP_DOWNLOAD_DIR esteja definido na Célula 1
# Por segurança, limpe a pasta de downloads antes de cada execução de teste
if os.path.exists(TEMP_DOWNLOAD_DIR):
    shutil.rmtree(TEMP_DOWNLOAD_DIR) # Remove a pasta e todo o seu conteúdo
time.sleep(1) # Pequena pausa para garantir que a pasta foi deletada
os.makedirs(TEMP_DOWNLOAD_DIR) # Recria a pasta vazia

driver = None
try:
    chrome_options = Options()
    if SELENIUM_HEADLESS: # Mantenha False para depurar o download visualmente
        chrome_options.add_argument("--headless")

    # Argumentos para otimização e estabilidade
    chrome_options.add_argument("--no-sandbox") # Essencial para Linux/Docker, pode ajudar no Windows
    chrome_options.add_argument("--disable-dev-shm-usage") # Reduz uso de RAM
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-gpu") # Pode ajudar em alguns ambientes headless

    # --- OPÇÕES CRÍTICAS PARA DOWNLOADS AUTOMÁTICOS ---
    # 1. Preferências de Download (prefs) - Dita onde e como o Chrome deve baixar
    prefs = {
        "download.default_directory": TEMP_DOWNLOAD_DIR, # Diretório onde os arquivos serão salvos
        "download.prompt_for_download": False,          # NÃO ABRIR CAIXA DE DIÁLOGO "SALVAR COMO"
        "download.directory_upgrade": True,             # Permite upgrade do diretório
        "plugins.always_open_pdf_externally": True,     # Abrir PDFs em visualizador externo

        # 2. Preferências de Segurança para Download - Essencial para desabilitar verificações
        "safeBrowse.enabled": False,                   # Desabilita a Navegação Segura por completo
        "safeBrowse.disable_download_protection": True,# Desabilita a proteção de download para URLs
        "profile.default_content_setting_values.automatic_downloads": 1, # Permite downloads automáticos (1=allow)
    }
    chrome_options.add_experimental_option("prefs", prefs)

    # 3. Argumentos adicionais que podem ajudar a burlar verificações
    chrome_options.add_argument("--disable-features=SafeBrowse") # Outra forma de desabilitar SafeBrowse
    chrome_options.add_argument("--allow-untrusted-downloads")     # Permite downloads de fontes não confiáveis
    chrome_options.add_argument("--disable-popup-blocking")        # Desabilita bloqueio de pop-ups (às vezes o prompt de download é tratado como tal)

    # Argumentos que podem ser necessários dependendo da página e do ambiente
    # chrome_options.add_argument("--disable-blink-features=AutomationControlled") # Para evitar detecção de automação
    # chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"]) # Para evitar a barra "Chrome está sendo controlado por software de automação"

    log_message("info", f"Inicializando o WebDriver Chrome com opções de download no diretório: {TEMP_DOWNLOAD_DIR}")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.set_page_load_timeout(REQUEST_TIMEOUT_SECONDS)
    log_message("info", "WebDriver inicializado com sucesso.")

except WebDriverException as e:
    log_message("error", f"WebDriverException ao inicializar o WebDriver. Verifique se o Chrome está instalado e o ChromeDriver é compatível. Erro: {e}")
    log_message("error", "Tente atualizar o Chrome ou a biblioteca webdriver-manager.")
    driver = None
except Exception as e:
    log_message("error", f"Erro inesperado ao inicializar o WebDriver: {e}")
    driver = None

SessionNotCreatedException: Message: Expected browser binary location, but unable to find binary in default location, no 'moz:firefoxOptions.binary' capability provided, and no binary flag set on the command line


In [ ]:
# Célula 3: Navegar para o Site e Inserir a Chave de Acesso

# ATENÇÃO: SUBSTITUA PELA SUA CHAVE DE ACESSO REAL PARA TESTE!
# Use uma chave válida para ver o resultado após a consulta.
test_key = "53250617244285000109550010008539971132114979" # <-- SUA CHAVE DE TESTE AQUI!

if driver:
    try:
        log_message("info", f"Navegando para: {MEUDANFE_WEB_URL}")
        driver.get(MEUDANFE_WEB_URL)

        # --- LOCALIZADORES COM BASE NO HTML QUE VOCÊ FORNECEU ---
        # Campo de input da chave de acesso (pelo placeholder)
        input_field_locator = (By.XPATH, "//input[@placeholder='Digite a CHAVE DE ACESSO']")
        
        # Botão "Buscar DANFE/XML" (pelo texto)
        consult_button_locator = (By.XPATH, "//button[contains(text(), 'Buscar DANFE/XML')]")

        log_message("info", f"Procurando campo de input da chave com {input_field_locator}")
        WebDriverWait(driver, REQUEST_TIMEOUT_SECONDS).until(
            EC.presence_of_element_located(input_field_locator)
        )
        input_element = driver.find_element(*input_field_locator)
        input_element.send_keys(test_key)
        log_message("info", f"Chave {test_key[:10]}... inserida no campo.")

        log_message("info", f"Procurando botão de consulta com {consult_button_locator}")
        WebDriverWait(driver, REQUEST_TIMEOUT_SECONDS).until(
            EC.element_to_be_clickable(consult_button_locator)
        )
        driver.find_element(*consult_button_locator).click()
        log_message("info", "Botão 'Buscar DANFE/XML' clicado. Aguardando resultados...")

        # Dê um tempo para a página carregar após o clique.
        # O ideal é usar WebDriverWait para um elemento que aparece na página de resultados.
        # Por enquanto, um sleep é um substituto temporário para depuração.
        time.sleep(5) 
        log_message("info", "Tempo de espera inicial concluído. Verificando novos elementos...")

    except TimeoutException:
        log_message("error", "Tempo limite excedido ao esperar por elemento na página inicial.")
    except NoSuchElementException as e:
        log_message("error", f"Elemento não encontrado na página inicial: {e}")
    except Exception as e:
        log_message("error", f"Erro na navegação/inserção da chave: {e}")

[2025-06-26 16:05:56] [INFO] Navegando para: https://www.meudanfe.com.br/
[2025-06-26 16:05:57] [INFO] Procurando campo de input da chave com ('xpath', "//input[@placeholder='Digite a CHAVE DE ACESSO']")
[2025-06-26 16:05:57] [INFO] Chave 5325061724... inserida no campo.
[2025-06-26 16:05:57] [INFO] Procurando botão de consulta com ('xpath', "//button[contains(text(), 'Buscar DANFE/XML')]")
[2025-06-26 16:05:57] [INFO] Botão 'Buscar DANFE/XML' clicado. Aguardando resultados...
[2025-06-26 16:06:02] [INFO] Tempo de espera inicial concluído. Verificando novos elementos...


In [ ]:
# Célula 4: Extrair XML e DANFE (AJUSTADA!)

xml_content = None
pdf_content = None
note_number = None

if driver:
    try:
        log_message("info", "Aguardando carregamento da página de resultados (/ver-danfe)...")
        # Espera até que a URL mude para /ver-danfe
        WebDriverWait(driver, REQUEST_TIMEOUT_SECONDS).until(
            EC.url_contains("/ver-danfe")
        )
        log_message("info", f"Página de resultados carregada: {driver.current_url}")

        # --- TENTAR BAIXAR O XML ---
        # No seu HTML, procure por um botão ou link que diga "Baixar XML" ou algo similar.
        # Ele deve estar na nova página '/ver-danfe'.
        # O HTML que você forneceu agora não mostra um botão de download XML,
        # mas sim um botão "Nova consulta". Você precisará INSPECIONAR a página /ver-danfe.

        # Cenário mais provável: Um botão de download direto para o XML
        # Exemplo: <button id="btnDownloadXml">Baixar XML</button>
        # Ou: <a href="caminho/para/xml.xml">Baixar XML</a>
        
        xml_download_locator = (By.XPATH, "//button[contains(text(), 'Baixar XML')]") # <--- AJUSTE ESTE LOCALIZADOR!
        # Tente outros seletores se não encontrar um botão com texto "Baixar XML"
        # Ex: (By.ID, "downloadXmlButton")
        # Ex: (By.CSS_SELECTOR, "a[href$='.xml']") # Para links que terminam com .xml

        log_message("info", f"Procurando botão/link de download de XML com localizador: {xml_download_locator}")
        WebDriverWait(driver, REQUEST_TIMEOUT_SECONDS).until(
            EC.element_to_be_clickable(xml_download_locator)
        )
        xml_download_element = driver.find_element(*xml_download_locator)
        
        log_message("info", "Botão/link de Baixar XML encontrado. Clicando...")
        xml_download_element.click() 

        # IMPORTANTE: Se o clique inicia um download, o Selenium não 'espera' pelo download.
        # Precisamos de uma lógica para verificar a pasta de downloads.
        time.sleep(3) # Dê um pequeno tempo para o download iniciar

        # Lógica para encontrar o arquivo XML mais recente na pasta de downloads
        # Ele pode ter um nome genérico como 'document.xml' ou 'download.xml' ou o número da chave.
        downloaded_xml_files = [f for f in os.listdir(TEMP_DOWNLOAD_DIR) if f.endswith('.xml')]
        
        if downloaded_xml_files:
            # Pega o arquivo XML mais recentemente modificado (provavelmente o que acabou de ser baixado)
            latest_xml_file_path = max([os.path.join(TEMP_DOWNLOAD_DIR, f) for f in downloaded_xml_files], key=os.path.getctime)
            log_message("info", f"Arquivo XML baixado encontrado: {latest_xml_file_path}")
            with open(latest_xml_file_path, 'rb') as f:
                xml_content = f.read()
            log_message("info", "Conteúdo XML lido do arquivo baixado.")
            
            # Opcional: Remover o arquivo depois de ler, para não acumular
            # os.remove(latest_xml_file_path)
            # log_message("debug", f"Arquivo temporário {latest_xml_file_path} removido.")
        else:
            log_message("error", "Nenhum arquivo XML encontrado na pasta de downloads após o clique.")
            # Se não encontrou, talvez o XML esteja na própria página como texto, ou a lógica de download é outra.
            # Você precisaria inspecionar o HTML da página /ver-danfe para confirmar.
            # Por enquanto, vamos simular se não encontrar para continuar o fluxo.
            xml_content = b"<NFe><infNFe><ide><nNF>000000000</nNF></ide></infNFe></NFe>"
            log_message("warning", "XML simulado, pois o download não foi detectado ou falhou.")


        # Extrair número da nota do XML (usando o XML obtido ou simulado)
        if xml_content:
            note_number = extract_note_number_from_xml(xml_content, log_message)
            if not note_number or note_number == "000000000":
                if len(key) >= 9:
                    note_number = key[-9:]
                else:
                    note_number = "UNKNOWN_NOTE"
                log_message("warning", f"Número da nota não extraído do XML. Usando: {note_number}")
        else:
            note_number = "UNKNOWN_NOTE" # Se nem o XML simulado foi setado


        # --- TENTAR BAIXAR O DANFE PDF ---
        # Repita a mesma lógica para o botão/link de "Baixar DANFE"
        danfe_download_locator = (By.XPATH, "//button[contains(text(), 'Baixar DANFE')]") # <--- AJUSTE ESTE LOCALIZADOR!
        # Tente outros seletores se não encontrar um botão com texto "Baixar DANFE"
        # Ex: (By.ID, "downloadDanfePdf")
        # Ex: (By.CSS_SELECTOR, "a[href$='.pdf']")

        log_message("info", f"Procurando botão/link de download de DANFE com localizador: {danfe_download_locator}")
        try:
            WebDriverWait(driver, REQUEST_TIMEOUT_SECONDS).until(
                EC.element_to_be_clickable(danfe_download_locator)
            )
            danfe_download_element = driver.find_element(*danfe_download_locator)
            
            log_message("info", "Botão/link de Baixar DANFE encontrado. Clicando...")
            danfe_download_element.click()

            time.sleep(3) # Dê um pequeno tempo para o download iniciar

            # Lógica para encontrar o arquivo PDF mais recente na pasta de downloads
            downloaded_pdf_files = [f for f in os.listdir(TEMP_DOWNLOAD_DIR) if f.endswith('.pdf')]
            if downloaded_pdf_files:
                latest_pdf_file_path = max([os.path.join(TEMP_DOWNLOAD_DIR, f) for f in downloaded_pdf_files], key=os.path.getctime)
                log_message("info", f"Arquivo DANFE PDF baixado encontrado: {latest_pdf_file_path}")
                with open(latest_pdf_file_path, 'rb') as f:
                    pdf_content = f.read()
                log_message("info", "Conteúdo DANFE PDF lido do arquivo baixado.")
                # Opcional: Remover o arquivo depois de ler
                # os.remove(latest_pdf_file_path)
            else:
                log_message("error", "Nenhum arquivo PDF encontrado na pasta de downloads após o clique.")
                pdf_content = b"%PDF-1.4\n% Conteudo PDF SIMULADO (Download falhou) \n%%EOF"
                log_message("warning", "PDF simulado, pois o download não foi detectado ou falhou.")

        except (TimeoutException, NoSuchElementException):
            log_message("warning", "Link de download de DANFE não encontrado ou não clicável. Usando PDF simulado.")
            pdf_content = b"%PDF-1.4\n% Conteudo PDF SIMULADO (Link not found) \n%%EOF"
        except Exception as e:
            log_message("error", f"Erro ao tentar baixar DANFE via link: {e}", exc_info=True)
            pdf_content = b"%PDF-1.4\n% Conteudo PDF SIMULADO (Erro Geral) \n%%EOF"


        log_message("info", f"Web scraping concluído para a nota: {note_number} (chave: {key[:10]}...).")
        
        print("\n--- Resultados do Scraper (Para Inspeção) ---")
        print(f"Número da Nota: {note_number}")
        print(f"XML Content (primeiros 200 chars): {xml_content[:200].decode('utf-8', errors='ignore')}...")
        print(f"PDF Content (tamanho): {len(pdf_content)} bytes")
        
    except TimeoutException:
        log_message("error", f"Tempo limite excedido ao esperar por elemento na página de resultados para a chave: {key}.")
    except NoSuchElementException as e:
        log_message("error", f"Elemento esperado não encontrado na página de resultados para a chave: {key}: {e}")
    except WebDriverException as e:
        log_message("error", f"Erro no WebDriver (navegador) para a chave: {key}: {e}", exc_info=True)
    except Exception as e:
        log_message("critical", f"Erro crítico e inesperado durante o scraping: {e}", exc_info=True)
    finally:
        # Importante: O driver.quit() será feito na próxima célula para permitir inspeção manual
        pass

[2025-06-26 16:06:02] [INFO] Aguardando carregamento da página de resultados (/ver-danfe)...
[2025-06-26 16:06:02] [INFO] Página de resultados carregada: https://www.meudanfe.com.br/ver-danfe
[2025-06-26 16:06:02] [INFO] Procurando botão/link de download de XML com localizador: ('xpath', "//button[contains(text(), 'Baixar XML')]")
[2025-06-26 16:06:02] [INFO] Botão/link de Baixar XML encontrado. Clicando...
[2025-06-26 16:06:05] [ERROR] Nenhum arquivo XML encontrado na pasta de downloads após o clique.
[2025-06-26 16:06:05] [WARNING] XML simulado, pois o download não foi detectado ou falhou.


TypeError: log_message() got an unexpected keyword argument 'exc_info'